In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/denny/Documents/mids/ev_charging'

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


2024-03-10 21:45:04.908044: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-10 21:45:04.908771: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-10 21:45:04.914890: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-10 21:45:04.999605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 21:45:06.180329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

TensorFlow version: 2.16.1


In [3]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_preprocessing import datetime_processing, userinput_processing, holiday_processing, create_x, create_y, create_wide_y
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
tf.random.set_seed(42)

In [4]:
# import os
# os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
# os.getcwd()

In [5]:
df = pd.read_parquet('data/sessions/caltech/').reset_index(drop=True)
df = datetime_processing(df)
df = userinput_processing(df)
df = holiday_processing(df)

In [6]:
df.head()

,_id,sessionID,stationID,spaceID,siteID,clusterID,connectionTime,disconnectTime,kWhDelivered,doneChargingTime,...,doneChargingTimeDay,userID,milesRequested,WhPerMile,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,kWhRequested,is_holiday
0,5bc9160af9af8b0dad3c0690,2_39_95_27_2018-05-01 13:23:18.374684,2-39-95-27,CA-319,2,39,2018-05-01 13:23:18+00:00,2018-05-02 01:53:41+00:00,13.410,2018-05-01 17:17:07+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
1,5bc9160af9af8b0dad3c0691,2_39_79_380_2018-05-01 13:54:33.599590,2-39-79-380,CA-489,2,39,2018-05-01 13:54:34+00:00,2018-05-01 23:15:35+00:00,8.618,2018-05-01 15:35:39+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
2,5bc9160af9af8b0dad3c0692,2_39_139_28_2018-05-01 14:46:04.441389,2-39-139-28,CA-303,2,39,2018-05-01 14:46:04+00:00,2018-05-02 00:54:07+00:00,13.969,2018-05-01 23:57:02+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
3,5bc9160af9af8b0dad3c0693,2_39_79_378_2018-05-01 14:56:28.908623,2-39-79-378,CA-326,2,39,2018-05-01 14:56:29+00:00,2018-05-01 21:52:45+00:00,6.914,2018-05-01 16:07:56+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
4,5bc9160af9af8b0dad3c0694,2_39_88_24_2018-05-01 15:14:50.443137,2-39-88-24,CA-314,2,39,2018-05-01 15:14:50+00:00,2018-05-01 22:51:30+00:00,13.785,2018-05-01 20:18:12+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False


In [7]:
df.stationID.value_counts()

stationID
2-39-139-28     1792
2-39-131-30     1399
2-39-129-17     1136
2-39-81-4550    1126
2-39-89-25      1086
2-39-125-21     1042
2-39-127-19     1022
2-39-138-29      994
2-39-79-379      932
2-39-123-23      908
2-39-79-380      892
2-39-78-365      862
2-39-91-437      846
2-39-79-377      820
2-39-130-31      789
2-39-78-366      759
2-39-95-27       737
2-39-88-24       694
2-39-79-383      683
2-39-124-22      665
2-39-79-378      645
2-39-78-363      615
2-39-128-18      612
2-39-79-381      587
2-39-79-376      581
2-39-78-360      578
2-39-90-26       567
2-39-126-20      561
2-39-92-436      558
2-39-78-367      533
2-39-78-361      529
2-39-79-382      506
2-39-92-442      503
2-39-78-364      458
2-39-91-441      452
2-39-78-362      452
2-39-90-440      432
2-39-95-444      396
2-39-88-438      332
2-39-89-439      330
2-39-123-557     247
2-39-83-386      200
2-39-138-566     175
2-39-124-558     149
2-39-130-564     136
2-39-126-560     124
2-39-128-562     120
2-3

In [8]:
def get_results(y_test, prediction):
    cm = confusion_matrix(y_test,prediction)
    make_classification_plot(cm)
    
    results = {'tpr': cm[1, 1]/np.sum(cm[1]),
               'fpr': cm[0,1]/np.sum(cm[0]),
               'accuracy': accuracy_score(y_test, prediction),
               'precision': precision_score(y_test, prediction),
               'recall': recall_score(y_test, prediction),
        'f1':f1_score(y_test,prediction)}
    return results

In [9]:
def make_classification_plot(cm):
    disp = ConfusionMatrixDisplay(cm)
    disp = disp.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
    plt.grid(False)
    plt.show()

In [10]:
results = {}

In [11]:
# create x and y 
start = '2018-05-01'
end = '2021-09-13'
spaceID = 'CA-303'
caiso_fp = 'data/caiso_dataset_5min.csv'
sun_fp = 'data/ct_sr_ss.csv'

In [ ]:
X = create_x(start, end, caiso_fp, sun_fp)
y = create_y(df, start, end, spaceID)

In [ ]:
X.head(30)

In [ ]:
print(f'charger CA-303 is available {np.round(y.mean()*100,3)}% of the time')

In [ ]:
#X.fillna(0.0, inplace=True)

In [ ]:
X.info()

In [ ]:
X.shape[0] == y.shape[0]

## Binary Forecasting

# LSTM

In [ ]:
# Create hold out test set
bin_numbers = pd.qcut(y['is_available'], q=10, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
print(f'the training data has an average availability of {np.round(y_train.mean()*100,3)}%')

In [ ]:
type(X_train)

In [ ]:
df_PCA = pd.DataFrame(X_train)
scaler = StandardScaler()
scaler.fit(df_PCA)
scaled_data = scaler.transform(df_PCA)
principal=PCA(n_components=10)
principal.fit(scaled_data)
print(principal.explained_variance_ratio_)

In [ ]:
train_scaler = StandardScaler()
test_scaler = StandardScaler()

In [ ]:
X_train_LSTM = pd.DataFrame(train_scaler.fit_transform(X_train))
X_test_LSTM = pd.DataFrame(test_scaler.fit_transform(X_test))

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(4, input_shape=(1, X_train_LSTM.shape[1])))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
X_train_LSTM = np.reshape(X_train_LSTM.values.astype(float), (X_train_LSTM.shape[0], 1, X_train_LSTM.shape[1]))
X_test_LSTM = np.reshape(X_test_LSTM.values.astype(float), (X_test_LSTM.shape[0], 1, X_test_LSTM.shape[1]))

In [ ]:
[print(i.shape, i.dtype) for i in lstm_model.inputs]
[print(o.shape, o.dtype) for o in lstm_model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in lstm_model.layers]

In [ ]:
X_train.shape

In [ ]:
y_train = y_train.drop('sessionID', axis=1) 
y_test = y_test.drop('sessionID', axis=1)

In [ ]:
lstm_model.fit(X_train_LSTM, y_train, epochs=10, batch_size=8, verbose=2)

In [ ]:
predictions = lstm_model.predict(X_test_LSTM)

In [ ]:
# MSE with CAISO
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

# XGBoost

In [ ]:
X_train = np.reshape(X_train.astype(float), (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test.astype(float), (X_test.shape[0], X_test.shape[1]))

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

In [ ]:
y_pred = predictions
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

## Multinomial Forecasting

# LSTM

In [ ]:
y = create_wide_y(df, start, end)

In [ ]:
y = y.sum(axis=1)

In [ ]:
y.head()

In [ ]:
# Create hold out test set
bin_numbers = pd.qcut(y, q=10, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
print(f'the training data has an average availability of {np.round(y_train.mean(),3)} spots available')

In [ ]:
train_scaler = StandardScaler()
test_scaler = StandardScaler()
X_train_LSTM = pd.DataFrame(train_scaler.fit_transform(X_train))
X_test_LSTM = pd.DataFrame(test_scaler.fit_transform(X_test))

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(4, input_shape=(1, X_train_LSTM.shape[1])))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
X_train_LSTM = np.reshape(X_train_LSTM.values.astype(float), (X_train_LSTM.shape[0], 1, X_train_LSTM.shape[1]))
X_test_LSTM = np.reshape(X_test_LSTM.values.astype(float), (X_test_LSTM.shape[0], 1, X_test_LSTM.shape[1]))

In [ ]:
[print(i.shape, i.dtype) for i in lstm_model.inputs]
[print(o.shape, o.dtype) for o in lstm_model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in lstm_model.layers]

In [ ]:
lstm_model.fit(X_train_LSTM, y_train, epochs=10, batch_size=8, verbose=2)

In [ ]:
predictions = lstm_model.predict(X_test_LSTM)

In [ ]:
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

In [ ]:
y_pred = pd.Series(predictions, index=X_test.index)

In [ ]:
y_pred = pd.Series(predictions, index=X_test.index).sort_index()
y_pred.plot()
y_test = y_test.sort_index()
y_test.plot()

In [ ]:
y_pred.hist()
y_test.hist()

In [ ]:
plt.scatter(y_test, predictions)

In [ ]:
plt.plot(y_test.index, y_test)

In [ ]:
plt.plot(y_test.index, predictions)

In [ ]:
y_test

# XGBoost

In [ ]:
X_train = np.reshape(X_train.astype(float), (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test.astype(float), (X_test.shape[0], X_test.shape[1]))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

In [ ]:
y_pred = le.inverse_transform(predictions)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

Denny quick run

In [12]:
# X = create_x(start, end, caiso_fp, sun_fp)
# y = create_y(df, start, end, spaceID)
create_x(df, start_date=sta)

,_id,sessionID,stationID,spaceID,siteID,clusterID,connectionTime,disconnectTime,kWhDelivered,doneChargingTime,...,doneChargingTimeDay,userID,milesRequested,WhPerMile,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,kWhRequested,is_holiday
0,5bc9160af9af8b0dad3c0690,2_39_95_27_2018-05-01 13:23:18.374684,2-39-95-27,CA-319,2,39,2018-05-01 13:23:18+00:00,2018-05-02 01:53:41+00:00,13.410000,2018-05-01 17:17:07+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
1,5bc9160af9af8b0dad3c0691,2_39_79_380_2018-05-01 13:54:33.599590,2-39-79-380,CA-489,2,39,2018-05-01 13:54:34+00:00,2018-05-01 23:15:35+00:00,8.618000,2018-05-01 15:35:39+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
2,5bc9160af9af8b0dad3c0692,2_39_139_28_2018-05-01 14:46:04.441389,2-39-139-28,CA-303,2,39,2018-05-01 14:46:04+00:00,2018-05-02 00:54:07+00:00,13.969000,2018-05-01 23:57:02+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
3,5bc9160af9af8b0dad3c0693,2_39_79_378_2018-05-01 14:56:28.908623,2-39-79-378,CA-326,2,39,2018-05-01 14:56:29+00:00,2018-05-01 21:52:45+00:00,6.914000,2018-05-01 16:07:56+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
4,5bc9160af9af8b0dad3c0694,2_39_88_24_2018-05-01 15:14:50.443137,2-39-88-24,CA-314,2,39,2018-05-01 15:14:50+00:00,2018-05-01 22:51:30+00:00,13.785000,2018-05-01 20:18:12+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31120,5bc92c03f9af8b0dc677ca2a,2_39_89_25_2018-07-19 15:51:18.244544,2-39-89-25,CA-315,2,39,2018-07-19 15:51:18+00:00,2018-07-19 23:28:02+00:00,7.334000,2018-07-19 18:41:04+00:00,...,19.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
31121,5bc92c03f9af8b0dc677ca2b,2_39_78_366_2018-07-19 15:57:19.401534,2-39-78-366,CA-323,2,39,2018-07-19 15:57:19+00:00,2018-07-19 22:49:56+00:00,1.336511,2018-07-19 15:56:21+00:00,...,19.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
31122,5bc92c03f9af8b0dc677ca2c,2_39_90_440_2018-07-19 15:57:56.698186,2-39-90-440,CA-500,2,39,2018-07-19 15:57:57+00:00,2018-07-20 01:14:15+00:00,5.998000,2018-07-19 18:36:53+00:00,...,19.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
31123,5bc92c03f9af8b0dc677ca2d,2_39_131_30_2018-07-19 16:01:23.524413,2-39-131-30,CA-305,2,39,2018-07-19 16:01:24+00:00,2018-07-20 00:03:48+00:00,4.718000,2018-07-19 17:33:44+00:00,...,19.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False


# Optuna

In [ ]:
import optuna 
import xgboost as xgb

In [ ]:
def objective(trial):
    
  bin_numbers = pd.qcut(y, q=10, labels=False, duplicates='drop')
  train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
  train_y = le.fit_transform(train_y)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  max_depth = trial.suggest_int("max_depth", 2, 10,step=2, log=False)
  n_estimators = trial.suggest_int("n_estimators", 200, 500,step=100, log=False)

  model = XGBClassifier(objective= 'multi:softprob',
                        learning_rate = learning_rate,
                        n_estimators = n_estimators,
                        max_depth = max_depth,
                        seed=42)

  model.fit(train_x, train_y)

  y_pred = le.inverse_transform(model.predict(valid_x))
  accuracy = accuracy_score(valid_y, y_pred)

  # Handle pruning based on the intermediate value.
  if trial.should_prune():
    raise optuna.exceptions.TrialPruned()

  trial.set_user_attr(key="best_model", value=model) # save model
  return accuracy

# callback function to save the best model as user attribute
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600, callbacks=[callback])

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
def lstm_objective(trial):
    
  bin_numbers = pd.qcut(y, q=10, labels=False, duplicates='drop')
  train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
  
  train_scaler = StandardScaler()
  test_scaler = StandardScaler()
  X_train_LSTM = pd.DataFrame(train_scaler.fit_transform(train_x))
  X_test_LSTM = pd.DataFrame(test_scaler.fit_transform(valid_x))
  
  # tunable params
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  n_layers = trial.suggest_int("n_layers", 1, 4)
  n_units = trial.suggest_int("n_units", 4, 64)
  activation = trial.suggest_categorical("activation", ["relu", "selu", "elu", "swish"])
  dropout_rate = trial.suggest_float("lstm_dropout", 0.0, 0.3)
  
  # create model
  x0 = tf.keras.Input(shape=(1, X_train_LSTM.shape[1]))

  lstm_units = np.zeros(n_layers, dtype=int)
  #lstm_units[0] = trial.suggest_int("n_units_L1", 4, 64)
  lstm = LSTM(n_units, input_shape=(1, X_train_LSTM.shape[1]), return_sequences=True)(x0)
  for i in range(1, n_layers):
    #lstm_units[i] = trial.suggest_int(f"n_units_L{i+1}", 4, 64)
    lstm = LSTM(n_units, return_sequences=True)(lstm)
    lstm = Dropout(dropout_rate)(lstm)

  #dense_units = lstm_units[-1]
  lstm = Dense(n_units, activation=activation)(lstm)
  lstm = Dense(1)(lstm)
  model = tf.keras.Model(inputs=x0, outputs=lstm)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')
  
  [print(i.shape, i.dtype) for i in model.inputs]
  [print(o.shape, o.dtype) for o in model.outputs]
  [print(l.name, l.input_shape, l.dtype) for l in model.layers]
  
  X_train_LSTM = np.reshape(X_train_LSTM.values.astype(float), (X_train_LSTM.shape[0], 1, X_train_LSTM.shape[1]))
  X_test_LSTM = np.reshape(X_test_LSTM.values.astype(float), (X_test_LSTM.shape[0], 1, X_test_LSTM.shape[1]))
  
  model.fit(X_train_LSTM, train_y, epochs=10, batch_size=8, verbose=2)
  
  y_pred = model.predict(X_test_LSTM)
  score = mean_squared_error(valid_y, y_pred.flatten())

  # Handle pruning based on the intermediate value.
  if trial.should_prune():
    raise optuna.exceptions.TrialPruned()

  trial.set_user_attr(key="best_model", value=model) # save model
  return score

# callback function to save the best model as user attribute
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(lstm_objective, n_trials=100, timeout=600, callbacks=[callback])

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
'''
def objective(trial):
    # Create hold out test set
    bin_numbers = pd.qcut(y, q=10, labels=False, duplicates='drop')
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
    train_y = le.fit_transform(train_y)
    
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "num_class": len(set(y)),
        "objective": "multi:softprob",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = le.inverse_transform(preds)#np.rint(preds)
    accuracy = accuracy_score(valid_y, pred_labels)
    return accuracy
'''